# Style-Transfer with Pytorch
Code taken from [NEURAL TRANSFER USING PYTORCH](https://pytorch.org/tutorials/advanced/neural_style_tutorial.html?highlight=style%20transfer).

## 1. Setup:

### 1.1. Import libs

In [11]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms

import torchvision.models as models

import copy

import numpy as np

### 1.2. Choose machine

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1.3. Define the desired size of the output image
Don't go too high 'cause this may stress your machine. Start small and work your way up, benchmarking your machine at the same time.

In [16]:
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

## 2. Load Image

In [ ]:
2.1. Rescale image 